# FACTORBOOK motifs processing SOP for FILER integration

## 1. Introduction

Factorbook is a transcription factor (TF)-centric repository of all ENCODE ChIP-seq datasets on TF-binding regions, as well as the rich analysis results of these data.

#### 1.1 Factorbook motifs:
Factorbook motifs were identified by applying MEME to the top 500 IDR thresholded ChIP-seq peaks from more than 3000 ENCODE ChIP-seq experiments. Five motifs were identified per experiment. These motifs were then filtered for quality using peak centrality and enrichment metrics. In total, there are 6,069 motifs available.

[Factorbook motif catalog](https://downloads.wenglab.org/factorbook-download/complete-factorbook-catalog.meme.gz)

[Associated metadata](https://storage.googleapis.com/gcp.wenglab.org/factorbook_chipseq_meme_motifs.tsv)

#### 1.2. Factorbook motif annotation for FILER

Factorbook motifs are downloaded and processed for FILER integration. This processing involves annotating the factorbook motifs by identifying overlaps with known motifs from the HOCOMOCO and JASPAR databases.

TOMTOM was used to compare Factorbook MEME motifs against the HOCOMOCO and JASPAR catalogs. From tomtom results, the motifs with the lowest q-value were selected as the most confident overlaps between Factorbook motifs and the known motifs in HOCOMOCO and JASPAR. These selected motifs were then filtered for the condition that the query sequence is contained within the target sequence, or vice versa.

The purpose of this SOP is to document the steps and processes involved in processing and annotating Facorbook motifs.

#### 1.3 Data processing workflow

The image below illustrates the entire workflow involved in processing and annotating factorbook motifs for FILER integration.

![title](/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/fb_workflow.png)



## 2.Requirements

### 2.1 ENCODE metadata:

ENCODE metadata is necessary to construct the download URL for each of the factorbook motifs. For example, factorbook motif ID (e.g., ENCFF674XTY_CMTBCTGGGARTTGTAG) is generated by combining the ENCODE BED file ID with the consensus sequence. 

#### 2.1.1 metadata download:
Select all the bed and bigBed tracks available in the ENCODE experiment matrix. Used the below selection criteria:

- Biosample: Organism --> Homo sapiens
- Quality: Status --> released, archived, revoked
- Analysis: Available file types --> All bed and bigBed file types
- Analysis: Genome Assembly --> hg19, GRCh38

The following URL is pre-selected to meet the above criteria on the ENCODE data portal:

[ENCODE metadata](https://www.encodeproject.org/search/?type=Experiment&status=released&perturbed=false&replicates.library.biosample.donor.organism.scientific_name=Homo+sapiens&perturbed=true&status=archived&status=revoked&files.file_type=bed+narrowPeak&files.file_type=bigBed+narrowPeak&files.file_type=bed+idr_ranked_peak&files.file_type=bed+broadPeak&files.file_type=bed+bed3%2B&files.file_type=bigBed+broadPeak&files.file_type=bigBed+bed3%2B&files.file_type=bed+bedRnaElements&files.file_type=bigBed+bedRnaElements&files.file_type=bed+tss_peak&files.file_type=bigBed+tss_peak&files.file_type=bed+bedMethyl&files.file_type=bigBed+bedMethyl&files.file_type=bed+bed9%2B&files.file_type=bed+bedGraph&files.file_type=bigInteract&files.file_type=bed+idr_peak&files.file_type=bigBed+idr_peak&files.file_type=bigBed+bed9%2B&files.file_type=bed+bed9&files.file_type=bigBed+bed9&files.file_type=bed+bedLogR&files.file_type=bigBed+bedLogR&files.file_type=bed+bed12&files.file_type=bigBed+bed12&files.file_type=bigBed+bed6%2B&files.file_type=bed+bed6%2B&files.file_type=bed+bedExonScore&files.file_type=bigBed+bedExonScore&files.file_type=bed+peptideMapping&files.file_type=bigBed+peptideMapping&files.file_type=bed+bed3&files.file_type=bigBed+bed3&files.file_type=bed+modPepMap&files.file_type=bed+pepMap&files.file_type=bigBed+modPepMap&files.file_type=bigBed+pepMap)

### 2.2 Tomtom: Motif comparison tool

Tomtom compares one or more motifs against a database of known motifs (e.g., JASPAR). 

[MEME suite V5.5.5](https://meme-suite.org/meme/doc/download.html)

### 2.3 Samtools

[Samtools V1.17](https://www.htslib.org/)

## 3. FILER data processing pipeline

The image below provides an overview of the data processing pipeline involved in factorbook motifs processing for FILER integration, outlining the inputs, outputs, and how each step is interconnected.


![title](/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/fb_pipeline.png)

### 3.1 Collect ENCODE experiment IDs from Factorbook motif catalog metadata

This script extracts ENCODE experiment IDs from the Factorbook motif metadata. These IDs are used in subsequent steps to retrieve and process the associated ChIP-seq data.

Script: step1-get_ENCODE_experiments.sh

Input: factorbook motif metadata - factorbook_chipseq_meme_motifs.tsv

Output: ENCODE_experimentIDs.txt

In [17]:
import os

def display_file_head(file_path):
    try:
        file_name = os.path.basename(file_path)  
        print(f"\n{file_name}:\n")  
        with open(file_path, 'r') as file:
            for _ in range(6): 
                print(file.readline().strip())
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

factorbook_metadata = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/factorbook_chipseq_meme_motifs.tsv"
encode_exp = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/ENCODE_experimentIDs.txt"

print("### Input Files ###")
display_file_head(factorbook_metadata)

print("\n### Output File ###")
display_file_head(encode_exp)


### Input Files ###

factorbook_chipseq_meme_motifs.tsv:

Animal	Biosample	Target	Identifier	Consensus	Sample Type	Information
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	AAAGAGGAASTGAAA	cell line	"inconsistent platforms,mild to moderate bottlenecking,moderate library complexity,mild to moderate bottlenecking,mild to moderate bottlenecking"
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	CCTCCTAGCCCTRACACACAGCTGGG	cell line	"inconsistent platforms,mild to moderate bottlenecking,moderate library complexity,mild to moderate bottlenecking,mild to moderate bottlenecking"
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	SCBSBCMSBBCCRVCCHSCSVKCCYKSVCC	cell line	"inconsistent platforms,mild to moderate bottlenecking,moderate library complexity,mild to moderate bottlenecking,mild to moderate bottlenecking"
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	GCSYGGGGCAAGTGACCGTGCGTGTAAAGG	cell line	"inconsistent platforms,mild to moderate bottlenecking,moderate library complexity,mild to moderate bottlenecking,mild to moderate

### 3.2 Process ENCODE metadata to map IDR thresholded peak BED file IDs for all ChIP-seq experiments

This script processes the ENCODE metadata and maps IDR thresholded peak BED file IDs to their corresponding ChIP-seq experiments, it then constructs Factorbook motif IDs by combining the ENCODE BED file ID with the consensus sequence for each experiment. 

Script: step2-bedtobigBed_mapping.py

Input: 
1. ENCODE metadata from section 2.1
2. factorbook motifs metadata from section 1.1
3. ENCODE experiment IDs from section 3.1

Output: bedtobigBed_mapping.txt

In [18]:
import os

def display_file_head(file_path):
    try:
        file_name = os.path.basename(file_path)  
        print(f"\n{file_name}:\n")  
        with open(file_path, 'r') as file:
            for _ in range(6): 
                print(file.readline().strip())
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

encode_metadata_file = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/filtered_metadata.txt"
encode_experiment_ids_file = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/ENCODE_experimentIDs.txt"
factorbook_motifs_metadata_file = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/factorbook_chipseq_meme_motifs.tsv"
mapping_output = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/bedtobigBed_mapping.txt"

print("### Input Files ###")
display_file_head(encode_metadata_file)
display_file_head(factorbook_motifs_metadata_file)
display_file_head(encode_experiment_ids_file)

print("\n### Output File ###")
display_file_head(mapping_output)


### Input Files ###

filtered_metadata.txt:

File accession	File type	File format type	Output type	Experiment accession	Biological replicate(s)	Derived from
ENCFF105HVH	bed	narrowPeak	IDR thresholded peaks	ENCSR228ELU	2	/files/ENCFF604PVE/, /files/ENCFF872HPF/, /files/ENCFF356LFX/
ENCFF214SNH	bed	narrowPeak	IDR thresholded peaks	ENCSR228ELU	1, 2	/files/ENCFF604PVE/, /files/ENCFF932FOK/, /files/ENCFF872HPF/, /files/ENCFF356LFX/
ENCFF204NXZ	bed	idr_ranked_peak	IDR ranked peaks	ENCSR228ELU	1, 2	/files/ENCFF932FOK/, /files/ENCFF872HPF/, /files/ENCFF604PVE/
ENCFF254RJL	bigBed	narrowPeak	IDR thresholded peaks	ENCSR228ELU	1, 2	/files/ENCFF214SNH/
ENCFF228MFI	bed	idr_ranked_peak	IDR ranked peaks	ENCSR228ELU	2	/files/ENCFF872HPF/, /files/ENCFF604PVE/

factorbook_chipseq_meme_motifs.tsv:

Animal	Biosample	Target	Identifier	Consensus	Sample Type	Information
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	AAAGAGGAASTGAAA	cell line	"inconsistent platforms,mild to moderate bottlenecking,moderate library com

### 3.3 Download Factorbook motifs

This script downloads factorbook motifs in BED format, by constructing download URLs using motif IDs from section 3.2

Script: step3-download_facrotbook_motifs.py

Input: bedtobigBed_mapping.txt from section 3.2

Output: binding sites for the motif ENCFF675PHY_STGATGCAABC in BED format

In [19]:
import os
import gzip

def display_file_head(file_path):
    try:
        file_name = os.path.basename(file_path)  
        print(f"\n{file_name}:\n")  
        
        if file_path.endswith('.gz'):
            with gzip.open(file_path, 'rt') as file:
                for _ in range(6):
                    print(file.readline().strip())
        else:
            with open(file_path, 'r') as file:
                for _ in range(6): 
                    print(file.readline().strip())
                    
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

mapping_file = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/bedtobigBed_mapping.txt"
downloaded_BED_file = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/fb_motifs/ENCFF675PHY_STGATGCAABC.gz"

print("### Input Files ###")
display_file_head(mapping_file)

print("\n### Output File ###")
display_file_head(downloaded_BED_file)


### Input Files ###

bedtobigBed_mapping.txt:

Animal	Biosample	Target	Experiment_ID	Experiment_ID	bigBed_Accession	BED_Accession	Consensus	Sample_Type	BED_Consensus
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	ENCSR437GBJ	ENCFF340KVJ	ENCFF002XEC	AAAGAGGAASTGAAA	cell line	ENCFF002XEC_AAAGAGGAASTGAAA
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	ENCSR437GBJ	ENCFF340KVJ	ENCFF002XEC	CCTCCTAGCCCTRACACACAGCTGGG	cell line	ENCFF002XEC_CCTCCTAGCCCTRACACACAGCTGGG
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	ENCSR437GBJ	ENCFF340KVJ	ENCFF002XEC	SCBSBCMSBBCCRVCCHSCSVKCCYKSVCC	cell line	ENCFF002XEC_SCBSBCMSBBCCRVCCHSCSVKCCYKSVCC
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	ENCSR437GBJ	ENCFF340KVJ	ENCFF002XEC	GCSYGGGGCAAGTGACCGTGCGTGTAAAGG	cell line	ENCFF002XEC_GCSYGGGGCAAGTGACCGTGCGTGTAAAGG
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	ENCSR437GBJ	ENCFF340KVJ	ENCFF002XEC	TGGACTTTGRACYYW	cell line	ENCFF002XEC_TGGACTTTGRACYYW

### Output File ###

ENCFF675PHY_STGATGCAABC.gz:

chr11	3057416	3057427	+	0.0529
chr6	6655191	6655202

### 3.4 Run Tomtom to compare factorbook motifs to HOCOMOCO and JASPAR known motifs and process the results

This script runs Tomtom on factorbook motifs against HOCOMOCO and JASPAR motif databases, then process the output to identify the best motif matches based on q-value and sequence overlap.

Script: step4-run_tomtom.py

Input: 
1. complete-factorbook-catalog.meme from section 1.1
2. HOCOMOCO v12 Core collection - H12CORE_meme_format.meme 
3. HOCOMOCO v12 core metadata - H12CORE_motifs.tsv
4. JASPAR 2022 Core collection - JASPAR2022_CORE_non-redundant_v2.meme 

Output: tomtom_mapping.txt

In [20]:
import os

def display_file_head(file_path):
    try:
        file_name = os.path.basename(file_path)  
        print(f"\n{file_name}:\n")  
        with open(file_path, 'r') as file:
            for _ in range(8): 
                print(file.readline().strip())
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

fb_meme = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/complete-factorbook-catalog.meme"
hoco_meme = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/db/H12CORE_meme_format.meme"
jaspar_meme = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/db/JASPAR2022_CORE_non-redundant_v2.meme"
hoco_metadata = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/H12CORE_motifs.tsv"

processed_results = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/tomtom_mapping.txt"

print("### Input Files ###")
display_file_head(fb_meme)
display_file_head(hoco_meme)
display_file_head(jaspar_meme)
display_file_head(hoco_metadata)

print("\n### Output File ###")
display_file_head(processed_results)


### Input Files ###

complete-factorbook-catalog.meme:

MEME version 4.5
ALPHABET= ACGT

MOTIF ENCSR437GBJ_AAAGAGGAASTGAAA
letter-probability matrix: alength= 4 w= 15 nsites= 132 E= 4.400e-97
0.5758 0.0000 0.2045 0.2197
0.4848 0.1288 0.2045 0.1818
0.5833 0.0000 0.1591 0.2576

H12CORE_meme_format.meme:

MEME version 4

ALPHABET= ACGT

strands: + -

Background letter frequencies
A 0.25 C 0.25 G 0.25 T 0.25

JASPAR2022_CORE_non-redundant_v2.meme:

MEME version 4

ALPHABET= ACGT

strands: + -

Background letter frequencies
A 0.25 C 0.25 G 0.25 T 0.25

H12CORE_motifs.tsv:

Motif	LOGO	Gene (human)	Gene (mouse)	Quality	Motif subtype	Quality	TF family	UniProt ID (human)	UniProt ID (mouse)
AHR.H12CORE.0.P.B		AHR	Ahr	B	0	P	PAS {1.2.5}	AHR_HUMAN	AHR_MOUSE
AHRR.H12CORE.0.P.C		AHRR	Ahrr	C	0	P	PAS {1.2.5}	AHRR_HUMAN	AHRR_MOUSE
ALX1.H12CORE.0.SM.B		ALX1	Alx1	B	0	S+M	Paired-related HD {3.1.3}	ALX1_HUMAN	ALX1_MOUSE
ALX3.H12CORE.0.SM.B		ALX3	Alx3	B	0	S+M	Paired-related HD {3.1.3}	ALX3_HUMAN	ALX3_MOUSE
A

### 3.5 Process factorbook motifs - Annotation and filtering intervals by p-value.

This script processes factorbook motifs by integrating metadata from ENCODE, HOCOMOCO, and JASPAR databases. It annotates factorbook motifs with transcription factor information and generates both p-value-filtered and unfiltered datasets based on FIMO p-values.

script: step5-motif_processing.py

Input:
1. Factorbook motifs from section 3.3
2. bedtobigBed_mapping.txt from section 3.2
3. tomtom_mapping.txt from section 3.4

Output:
1. fb_motifs_filtered - annotated intervals filtered for p-value < 1e-4.
2. fb_motifs_processed - annotated intervals

In [14]:
import os
import gzip

def display_file_head(file_path):
    try:
        file_name = os.path.basename(file_path)  
        print(f"\n{file_name}:\n")  
        
        if file_path.endswith('.gz'):
            with gzip.open(file_path, 'rt') as file:
                for _ in range(6):
                    print(file.readline().strip())
        else:
            with open(file_path, 'r') as file:
                for _ in range(6): 
                    print(file.readline().strip())
                    
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

fb_motif = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/fb_motifs/ENCFF003ZLG_TYTCASTTCCTCWTT.gz"
encode_mapping_file = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/bedtobigBed_mapping.txt"
tomtom_mapping = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/tomtom_mapping.txt"

unfiltered_output = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/fb_motifs_processed/ENCFF003ZLG_TYTCASTTCCTCWTT_sorted.bed.gz" 

print("### Input Files ###")
display_file_head(fb_motif)
display_file_head(encode_mapping_file)
display_file_head(tomtom_mapping)


print("\n### Output File ###")
display_file_head(unfiltered_output)


### Input Files ###

ENCFF003ZLG_TYTCASTTCCTCWTT.gz:

chr15	77406322	77406337	-	0.00161
chr6	43684212	43684227	-	0.00161
chr4	134740490	134740505	-	0.00161
chr4	88859044	88859059	-	0.00161
chr2	172437883	172437898	-	0.00161
chr15	77406322	77406337	-	0.00161

bedtobigBed_mapping.txt:

Animal	Biosample	Target	Experiment_ID	Experiment_ID	bigBed_Accession	BED_Accession	Consensus	Sample_Type	BED_Consensus
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	ENCSR437GBJ	ENCFF340KVJ	ENCFF002XEC	AAAGAGGAASTGAAA	cell line	ENCFF002XEC_AAAGAGGAASTGAAA
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	ENCSR437GBJ	ENCFF340KVJ	ENCFF002XEC	CCTCCTAGCCCTRACACACAGCTGGG	cell line	ENCFF002XEC_CCTCCTAGCCCTRACACACAGCTGGG
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	ENCSR437GBJ	ENCFF340KVJ	ENCFF002XEC	SCBSBCMSBBCCRVCCHSCSVKCCYKSVCC	cell line	ENCFF002XEC_SCBSBCMSBBCCRVCCHSCSVKCCYKSVCC
Homo sapiens	GM12878	NFATC3	ENCSR437GBJ	ENCSR437GBJ	ENCFF340KVJ	ENCFF002XEC	GCSYGGGGCAAGTGACCGTGCGTGTAAAGG	cell line	ENCFF002XEC_GCSYGGGGCAAGTGACCGTGCGTGT

In [15]:
def display_file(filepath):
    with open(filepath, 'r') as file:
        for line in file:
            print(line.strip())

header_file = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/bed17_column_description.tsv"

print('### BED file columns\n')

display_file(header_file)

### BED file columns

#COLUMN	DESCRIPTION

chrom	Chromosome in chrXXX notation (e.g., chr1, chr2)
chromStart	Start position of the binding site. 0-based coordinate system
chromEnd	End position of the binding site.
hocomoco_jaspar_TF_name	Transcription factor names from HOCOMOCO and JASPAR databases. If both are available, they are comma-separated
FB_score	FIMO p-value
strand	Strand (+ for forward, - for reverse)
binding_sequence	Strand specific nucleotide sequence that binds to the transcription factor, extracted from the reference genome using the binding site coordinates (reverse complemented for '-' starnd)
FB_pwm_ID	Factorbook motif Position Weight Matrix (PWM) ID
FB_consensus_sequence	Consensus sequence representing the most common nucleotides at each position within the Factorbook motif
FB_ChIP-Seq_target	Target of the ChIP-Seq experiment
HOCOMOCO_pwm_ID	HOCOMOCO motif Position Weight Matrix (PWM) ID
HOCOMOCO_Target_consensus	Consensus sequence from the HOCOMOCO database represen

### 3.6 Factorbook motif probability matrix

This script processes the factorbook motifs MEME catalog and writes each motif to a separate MEME file.

Script: step6-pwm-proecssing.py

Input: complete-factorbook-catalog.meme.gz from section 1.1

Output: ENCSR444LIN_CTTTGAAR.meme

In [20]:
import os

def display_file_head(file_path):
    try:
        file_name = os.path.basename(file_path)  
        print(f"\n{file_name}:\n")  
        with open(file_path, 'r') as file:
            for _ in range(20): 
                print(file.readline().strip())
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")

fb_meme = "/mnt/ebs/jackal/FILER2/FILER2-production/FACTORBOOK/complete-factorbook-catalog.meme"

print('### complete factorbook catalog')

display_file_head(fb_meme)

### complete factorbook catalog

complete-factorbook-catalog.meme:

MEME version 4.5
ALPHABET= ACGT

MOTIF ENCSR437GBJ_AAAGAGGAASTGAAA
letter-probability matrix: alength= 4 w= 15 nsites= 132 E= 4.400e-97
0.5758 0.0000 0.2045 0.2197
0.4848 0.1288 0.2045 0.1818
0.5833 0.0000 0.1591 0.2576
0.2500 0.0682 0.5758 0.1061
0.6515 0.0758 0.1742 0.0985
0.2348 0.0000 0.7652 0.0000
0.2045 0.0076 0.7879 0.0000
0.9167 0.0000 0.0682 0.0152
0.9773 0.0076 0.0000 0.0152
0.1742 0.3788 0.4470 0.0000
0.2424 0.0152 0.0379 0.7045
0.0758 0.0000 0.9242 0.0000
0.9167 0.0000 0.0530 0.0303
0.6742 0.0000 0.2803 0.0455
0.7727 0.0530 0.1288 0.0455
